In [118]:
from selenium.webdriver.common.keys import Keys
# import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from cdp_backend.pipeline import ingestion_models
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd

In [42]:
# Get basic council memeber information
city_council = pd.DataFrame()
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
council_url = 'https://www.minneapolismn.gov/government/city-council/'
driver.get(council_url)
cards = driver.find_elements(By.CSS_SELECTOR, '#image-text-link-50242 a.card-link')
len(cards)
for card in cards:
    driver.get(card.get_attribute('href'))
    position, name = driver.find_element(By.CSS_SELECTOR, '.masthead--title').text.split(' - ')
    role = driver.find_element(By.CSS_SELECTOR, 'a.contact span.title').text
    city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
    driver.back()
driver.quit()

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\3421782549.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from

In [43]:
city_council

,name,position,role
0,Elliott Payne,Ward 1,Council Member
1,Robin Wonsley,Ward 2,Council Member
2,Michael Rainville,Ward 3,Council Member
3,LaTrisha Vetaw,Ward 4,Council Member
4,Jeremiah Ellison,Ward 5,Council Member
5,Jamal Osman,Ward 6,Council Member
6,Lisa Goodman,Ward 7,Council Member
7,Andrea Jenkins,Ward 8,Council President
8,Jason Chavez,Ward 9,Council Member
9,Aisha Chughtai,Ward 10,Council Member


In [56]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\4143411418.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [129]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)
dt.isoformat()
dt.year, dt.month, dt.day
# video_link = driver.find_element(by='id', value='VideoIcon_0_0')
# video_link.click()
# video = WebDriverWait(driver, 4).until(
#     ec.presence_of_element_located((By.CSS_SELECTOR, '.close span'))
# )

# link = WebDriverWait(driver, 2).until(
#     ec.presence_of_element_located((By.CSS_SELECTOR, 'iframe'))
# )
# sessions.append(
#     ingestion_models.Session(
#         # Get Video Link:
#         video_uri=link.get_attribute('src'),
#         session_index=0,
#         session_datetime=dt
#     )
# )

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\200673377.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


ValueError: not enough values to unpack (expected 5, got 1)

In [137]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)
dt.isoformat()
dt.year, dt.month, dt.day


# Inspecting each minute_item:
li = driver.find_elements(by='css selector', value='li.customLineHeight')
event_minutes_items = []
month_convert = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}
for list in li[:6]:
    try:
        # try:
        if list.find_element(By.CSS_SELECTOR, 'a').get_attribute('class') == 'hrefnowrap':
            link = list.find_element(By.CSS_SELECTOR, 'a.hrefnowrap')
            print(link)
            # driver.get(link.get_attribute('href'))
            # print('a')

            
            # driver.back()
            
            link.click()
            before = driver.window_handles[0]
            after = driver.window_handles[1]
            driver.switch_to.window(after)
            element = WebDriverWait(driver, 2).until(
                ec.presence_of_element_located((By.CSS_SELECTOR, 'body'))
            )
            
            #以下准备uncomment
            
            minutes_item = element.find_element(By.CSS_SELECTOR, '.togglec .ng-binding')
            name = minutes_item.text.replace('This link open a new window\n', '')
            title = element.find_element(By.CSS_SELECTOR, '.legisDetails-tog p').text
            m_type = element.find_element(By.CSS_SELECTOR, '.togglec h5').text.split(' ')[0]
            decision = element.find_element(By.CSS_SELECTOR, 'span.detail_Status').text
            
            tables = element.find_elements(By.CSS_SELECTOR, '.col_full.nobottommargin')
            for table in tables:
                if 'Legislative History' in table.text:
                    rows = table.find_elements(By.CSS_SELECTOR, 'table tr')
                    rows = rows[1:-1]
                    for row in rows:
                        vote_date = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text #.split(' ')
                        vote_date = vote_date.replace(vote_date[:3], month_convert[vote_date[:3]])
                        vote_date = datetime.strptime(vote_date, "%B %d, %Y")
                        if (vote_date.year, vote_date.month, vote_date.day) == (dt.year, dt.month, dt.day):
                            vote_link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a')
                            vote_link.click()
                            
                            element = WebDriverWait(driver, 2).until(
                                ec.visibility_of_element_located((By.CLASS_NAME, '.vote-table'))
                            )
                            # print(element.text)
                            voters = driver.find_element(By.CSS_SELECTOR, 'vote-row')
                            print(voters.text)
                            # print(len(voters))
                            # print(voters[0])
                            # print(voters[0].text)
                            # for voter in voters:
                            #     print(voter.text)
                            break

            driver.switch_to.window(before)
        else:
            try:
                model = ingestion_models.EventMinutesItem(
                    minutes_item = ingestion_models.MinutesItem(
                        list.find_element(by='css selector', value='.custom-rcaview-action-review').text
                    ),
                    decision = list.find_element(by='css selector', value='.col-md-12 b').text.replace('Action Taken: ', '')
                )
                event_minutes_items.append(model)
            except (selenium.common.exceptions.NoSuchElementException):
                continue
        # except (selenium.common.exceptions.StaleElementReferenceException):
        #     print('error when requesting a new page')
        #     driver.quit()
    except (selenium.common.exceptions.NoSuchElementException):
        continue
# event_minutes_items

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\2044126189.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


<selenium.webdriver.remote.webelement.WebElement (session="f48bee263590bd5f488af1d702abaac7", element="17f603a0-eb2c-491d-ae49-3be4162066d6")>



In [136]:
driver.quit()